<a href="https://colab.research.google.com/github/arva29/AMDproject/blob/main/UkraineWarTweetsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this is meant to be run on google colab

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7") #SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [ ]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 12.6G/12.6G [01:19<00:00, 179MB/s]
100% 12.6G/12.6G [01:19<00:00, 171MB/s]


In [ ]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0819_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0820_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0821_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0822_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0823_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0824_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0825_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0826_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0827_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0828_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0829_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0830_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0831_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0901_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0902_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0903_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

## **Datasets extraction**

### *Single day dataset*

In [ ]:
import csv
import gzip

data_list = []

with gzip.open('1008_UkraineCombinedTweetsDeduped.csv.gzip', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_list.append(row)

rdd = sc.parallelize(data_list)

### *Single day dataset (5000 rows)* 


In [ ]:
import csv
import gzip

data_list = []
count = 0

with gzip.open('1008_UkraineCombinedTweetsDeduped.csv.gzip', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        count += 1
        data_list.append(row)
        if(count >= 5000): break

rdd = sc.parallelize(data_list)

### *10 day dataset*

In [ ]:
data_list = []

for i in range(1, 9):
  with gzip.open('100' + str(i) + '_UkraineCombinedTweetsDeduped.csv.gzip', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_list.append(row)

rdd = sc.parallelize(data_list)

### Test rdd

In [ ]:
rdd.count()

86471

In [ ]:
rdd.first()

OrderedDict([('', '130404'),
             ('userid', '1603709857'),
             ('username', 'trajanpublisher'),
             ('acctdesc',
              "We are Trajan Media, owner of Canada's premier numismatic and philatelic publications –\xa0Canadian Coin News and Canadian Stamp News –\xa0plus Coin & Stamp Supplies."),
             ('location', 'Canada'),
             ('following', '190'),
             ('followers', '916'),
             ('totaltweets', '7215'),
             ('usercreatedts', '2013-07-18 15:23:57'),
             ('tweetid', '1578535646146199554'),
             ('tweetcreatedts', '2022-10-08 00:00:00'),
             ('retweetcount', '1'),
             ('text',
              'Canada joined 15 countries whose flags grace the obverse of Ukraine’s first commemorative coin issued since the beginning of Russia’s renewed invasion this February.\n\nRead more: https://t.co/vRhROceH9x\n\n#numismatics #Ukraine️ #collectcoins'),
             ('hashtags',
              "[{'text':

Tokenize tweet by removing puntuation and stop words in order to have only meaningful words to understand the meaning of the tweet. I also put all the words lower case and remove the link at the end of the tweets.

## Tokenization

In [ ]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 29.6 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=9e66a241a0ca5af8d232bf2e17489fc1496d85f405e93ed80f537321ffffa5e3
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from cleantext import clean

nltk.download('stopwords')
stopw = set(stopwords.words('english'))

tokenizer = TweetTokenizer()

# Without stopwords
#tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation and word not in stopw]

# With stopwords
tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import numpy as np

def sort(list):
  list.sort()
  return list

en_tweets = rdd.filter(lambda x: x.get("language")=='en').map(lambda x: (x, x.get("text")))

tweets_tokenized = en_tweets.map(lambda x: (x[0].get("tweetid"), tokenize(x[1])))

tweets_tokenized.first()

('1578535646146199554',
 ['canada',
  'joined',
  '15',
  'countries',
  'whose',
  'flags',
  'grace',
  'the',
  'obverse',
  'of',
  "ukraine's",
  'first',
  'commemorative',
  'coin',
  'issued',
  'since',
  'the',
  'beginning',
  'of',
  "russia's",
  'renewed',
  'invasion',
  'this',
  'february',
  'read',
  'more',
  '#numismatics',
  '#ukraine',
  '#collectcoins'])

## *K*-Shingles

In [ ]:
K = int(input("Enter the number K for k-grams dimension: "))

def build_shingles(tokens, K):
  shingles = list()
  tmp = ""

  for i in range(len(tokens)):
    tmp = tokens[i]
    if((len(tokens) - (i)) < K):
      break;
    for k in range(K - 1):
      tmp += " " + tokens[i + (k + 1)]
    shingles.append(tmp)

  return shingles

shingles_rdd = tweets_tokenized.map(lambda x: (x[0], sort(build_shingles(x[1], K))))
shingles_rdd.count()

Enter the number K for k-grams dimension: 3


3006

Cast from list to set and then to list again in order to remove all the duplicate elements

In [ ]:
shingles_bag = sort(list(set(shingles_rdd.flatMap(lambda x: x[1]).collect())))
shingles_bag[0:10]

['##osce #un #ukrainerussiawar',
 '#1001daystoendwars #nuclearwar #norway',
 '#1004daywithjeonghan #crypto #kyiv',
 '#100daysofcode #nodejs #programmer',
 '#100daysofcode #programming #linux',
 '#100daysofcode #womenwhocode #nodejs',
 '#100devs #russia #100daysofcodechallenge',
 '#1776rm #trumpmeltdown #trumpstolethedocs',
 '#1776rm via @youtube',
 '#1800s #brazil #russia']

I use integers from 0 to *n* (with n equal to the length of shgles_bag) to store the shingles list

In [ ]:
shingles_bag_id = list(range(0, len(shingles_bag)))
len(shingles_bag_id)

56990

### One-hot enconding

In [ ]:
def convertShingles(shingles, BoS):
  listToReturn = [0]*len(BoS)
  start_index = 0;

  for s in shingles:
    for i in range(start_index, len(BoS)):
      if(s == BoS[i]):
        listToReturn[i] = 1
        start_index = i
        break;

  return listToReturn

shingles_int_rdd = shingles_rdd.map(lambda x: (x[0], convertShingles(x[1], shingles_bag)))
#shingles_int_rdd.first()

### List of positions

In [ ]:
def convertShingles(shingles, BoS):
  listToReturn = list()
  start_index = 0;

  for s in shingles:
    for i in range(start_index, len(BoS)):
      if(s == BoS[i]):
        listToReturn.append(i)
        start_index = i
        break;

  return listToReturn

shingles_int_rdd = shingles_rdd.map(lambda x: (x[0], convertShingles(x[1], shingles_bag)))
shingles_int_rdd.collect()
shingles_int_rdd.persist()

PythonRDD[4] at collect at <ipython-input-13-92a8e7ca14c7>:15

Hashing function for permutation

In [ ]:
def generatePermutation(arrayToHash, a, b):
  arrayToReturn = arrayToHash.copy()
  
  for i in range(len(arrayToHash)):
    arrayToReturn[i] = (a * arrayToHash[i] + b) % len(arrayToHash)

  return arrayToReturn

n = int(input("Insert number n of permutation desired: "))
permutation_matrix = list()

for i in range(n):
  #Generate 2 random numbers in order to have different hashing by randomly changing only these two numbers
  a = np.random.randint(1000) 
  b = np.random.randint(1000)
  #print(a, b)

  permutation_matrix.append(generatePermutation(shingles_bag_id, a, b))
  #print(permutation_matrix[i][:30])

Insert number n of permutation desired: 100


### For one-hot encoding

In [ ]:
def calculateMinhashSignature(perm_mtx, shingle_array):
  sig_column = [np.inf] * len(perm_mtx)

  for i in range(len(shingle_array)):
    if(shingle_array[i] == 1):
      for j in range(len(sig_column)):
        if(perm_mtx[j][i] < sig_column[j]):
          sig_column[j] = perm_mtx[j][i]

  return sig_column

minhash_sig_rdd = shingles_int_rdd.map(lambda x: (x[0], calculateMinhashSignature(permutation_matrix, x[1])))
minhash_sig_rdd.collect()
minhash_sig_rdd.persist()

PythonRDD[9] at collect at <ipython-input-20-605f355825c7>:13

### For other method

In [ ]:
def calculateMinhashSignature(perm_mtx, shingle_array):
  sig_column = [np.inf] * len(perm_mtx)

  for i in shingle_array:
    for j in range(len(sig_column)):
      if(perm_mtx[j][i] < sig_column[j]):
        sig_column[j] = perm_mtx[j][i]

  return sig_column

minhash_sig_rdd = shingles_int_rdd.map(lambda x: (x[0], calculateMinhashSignature(permutation_matrix, x[1])))
minhash_sig_rdd.collect()
minhash_sig_rdd.persist()

PythonRDD[5] at collect at <ipython-input-15-e87837df734b>:12

In [ ]:
b = int(input("Enter into how many bands you want to divide the " + str(n) +" permutations: "))

def band_splitting(tweet_id, arrayToSplit, b, n):
  tmp = list()
  arrayToReturn = list()
  band_id = 1

  r = int(n/b)
  i = 0

  while i < len(arrayToSplit):
    for j in range(r):
      tmp.append(arrayToSplit[i + j])
    
    i += r
    arrayToReturn.append((tweet_id, (band_id, tmp.copy())))
    band_id += 1
    tmp.clear()

  return arrayToReturn

sig_bands_rdd = minhash_sig_rdd.flatMap(lambda x: band_splitting(x[0], x[1], b, n))
sig_bands_rdd.take(100)

Enter into how many bands you want to divide the 100 permutations: 20


[('1578535646146199554', (1, [591, 1487, 1593, 863, 121])),
 ('1578535646146199554', (2, [6517, 8762, 228, 758, 8856])),
 ('1578535646146199554', (3, [1603, 515, 2496, 3595, 2259])),
 ('1578535646146199554', (4, [2055, 1891, 1668, 766, 1654])),
 ('1578535646146199554', (5, [264, 2787, 2339, 1331, 3360])),
 ('1578535646146199554', (6, [1298, 5980, 1443, 1126, 2809])),
 ('1578535646146199554', (7, [3837, 753, 511, 1363, 24])),
 ('1578535646146199554', (8, [930, 4252, 3411, 8151, 5212])),
 ('1578535646146199554', (9, [2246, 649, 2213, 374, 3699])),
 ('1578535646146199554', (10, [4934, 1282, 1795, 195, 857])),
 ('1578535646146199554', (11, [2026, 902, 9287, 4382, 1586])),
 ('1578535646146199554', (12, [5256, 3432, 904, 2474, 63])),
 ('1578535646146199554', (13, [925, 50, 3776, 821, 3179])),
 ('1578535646146199554', (14, [4478, 5502, 973, 843, 880])),
 ('1578535646146199554', (15, [3154, 5463, 844, 477, 703])),
 ('1578535646146199554', (16, [1222, 77, 4022, 2501, 4673])),
 ('157853564614619

In [ ]:
import hashlib

def hashArray(array):
  col_str = ' '.join(str(x) for x in array)

  return hash(col_str)

sig_grouped_rdd = sig_bands_rdd.map(lambda x: ((hashArray(x[1][1]), x[1][0]), x[0])).groupByKey().map(lambda x: (x[0], list(x[1])))
sig_grouped_rdd.collect()
sig_grouped_rdd.persist()

PythonRDD[11] at collect at <ipython-input-17-cfb226be695e>:9

In [ ]:
sig_grouped_rdd.filter(lambda x: len(x[1]) > 1).count()

1581

In [ ]:
sig_grouped_rdd.filter(lambda x: len(x[1]) > 1).collect()

[((-2750236260124232605, 8), ['1578535802534907904', '1578558087149285377']),
 ((-3532058187555594918, 19), ['1578535802534907904', '1578558087149285377']),
 ((8314570067702400201, 2), ['1578535891491954688', '1578554508342272000']),
 ((1267130330405097971, 3), ['1578535891491954688', '1578554508342272000']),
 ((-1783381037243443337, 5), ['1578535891491954688', '1578554508342272000']),
 ((4177429762574781957, 6), ['1578535891491954688', '1578554508342272000']),
 ((2762843017660440698, 15), ['1578535891491954688', '1578554508342272000']),
 ((-7730412902247045805, 18), ['1578535891491954688', '1578554508342272000']),
 ((-106634797599998402, 20), ['1578535891491954688', '1578554508342272000']),
 ((4227300581378564294, 15),
  ['1578535911444254720',
   '1578551010993528832',
   '1578566108025479168',
   '1578596308159537152']),
 ((-5415246452232365977, 19),
  ['1578535911444254720', '1578551010993528832', '1578566108025479168']),
 ((-5995808664921660276, 2),
  ['1578535992784433153', '1578

In [45]:
def array_splitting(id_array):
  listToReturn = list()

  for id in id_array:
    id_array.remove(id)
    listToReturn.append((id, id_array))

  return listToReturn

def removeDuplicate(x, y):

  return list(set(x + y))
  

similar_tweets_rdd = sig_grouped_rdd.filter(lambda x: len(x[1]) > 1).flatMap(lambda x: array_splitting(x[1])).reduceByKey(lambda x,y: removeDuplicate(x , y))

similar_tweets = similar_tweets_rdd.collect()
similar_tweets

[('1578535911444254720', ['1578596308159537152', '1578551010993528832']),
 ('1578566300993191936', ['1578551804136022017']),
 ('1578550272036831232',
  ['1578539136327090177', '1578552081853468672', '1578556256486567936']),
 ('1578556256486567936', ['1578539136327090177', '1578552081853468672']),
 ('1578549227592249344',
  ['1578547601812901888',
   '1578552081853468672',
   '1578539136327090177',
   '1578556256486567936']),
 ('1578536646789988352', ['1578539867780571136']),
 ('1578538642573623297', ['1578548099496091649']),
 ('1578539116567752705', ['1578602466098176000']),
 ('1578580015687548928', ['1578593851627171841', '1578557770348957697']),
 ('1578540238699319296',
  ['1578556675644043266',
   '1578554847279779840',
   '1578544601623068673',
   '1578544639787040768',
   '1578544816786657280',
   '1578555041400913921',
   '1578546828684894208']),
 ('1578544816786657280',
  ['1578556675644043266',
   '1578554847279779840',
   '1578544639787040768',
   '1578555041400913921',
   '15

In [50]:
rdd.filter(lambda x: x.get("tweetid") in similar_tweets[0][1]).map(lambda x: x.get("text")).collect()

["#odessa #odesa #ukraine #одесса\nNow: 13.9°C\nToday's Min: 13.9°C at 03:43:42\nToday's Max: 15.3°C at 00:00:00\nMonth's Min: 10.7°C at 2022-10-04 06:33:41\nMonth's Max: 24.2°C at 2022-10-01 13:01:39",
 "#odessa #odesa #ukraine #одесса\nNow: 13.2°C\nToday's Min: 13.3°C at 06:24:25\nToday's Max: 15.3°C at 00:00:00\nMonth's Min: 10.7°C at 2022-10-04 06:33:41\nMonth's Max: 24.2°C at 2022-10-01 13:01:39"]

In [60]:
count = 0

for tweet, listSimilarTweets in similar_tweets:
  count += 1
  print("--- TWEET ID: " + tweet + " ---")
  listSimilarTweets.insert(0, tweet)
  [print(" -  " + i) for i in (rdd.filter(lambda x: x.get("tweetid") in listSimilarTweets).map(lambda x: x.get("text").replace('\n', '')).collect())]
  print("\n")

  if count == 40:
    break


--- TWEET ID: 1578535911444254720 ---
 -  #odessa #odesa #ukraine #одессаNow: 14.1°CToday's Min: 14.2°C at 02:33:24Today's Max: 15.3°C at 00:00:00Month's Min: 10.7°C at 2022-10-04 06:33:41Month's Max: 24.2°C at 2022-10-01 13:01:39
 -  #odessa #odesa #ukraine #одессаNow: 13.9°CToday's Min: 13.9°C at 03:43:42Today's Max: 15.3°C at 00:00:00Month's Min: 10.7°C at 2022-10-04 06:33:41Month's Max: 24.2°C at 2022-10-01 13:01:39
 -  #odessa #odesa #ukraine #одессаNow: 13.2°CToday's Min: 13.3°C at 06:24:25Today's Max: 15.3°C at 00:00:00Month's Min: 10.7°C at 2022-10-04 06:33:41Month's Max: 24.2°C at 2022-10-01 13:01:39


--- TWEET ID: 1578566300993191936 ---
 -  #Live: CCTV: Day 226 The Ukraine War, (multiple views) #Ukraine #Russia #Lyman #Kiev #Kyiv #Kherson #Zaporozhye #Odessa #Mykolayiv #Kharkiv #Russia #UkraineRussiaWar #ukraineWar #NATO #WagnerGroup #LVIV #DNIPRO #DPR #Putin #BidensBlunder  https://t.co/0DqA2Z2TE8
 -  #LiveStreaming : CCTV: Day 226 The Ukraine War, (multiple views) #Ukrain

# DEBUG

In [ ]:
def generatePermutation(arrayToHash, a, b):
  arrayToReturn = arrayToHash.copy()
  
  for i in range(len(arrayToHash)):
    arrayToReturn[i] = (a * arrayToHash[i] + b) % len(arrayToHash)

  #print(arrayToHash, a, b, arrayToReturn)

  return arrayToReturn
  
def calculateMinhashSignature(perm_mtx, shingle_array):
  sig_column = [np.inf] * len(perm_mtx)

  for i in range(len(shingle_array)):
    if(shingle_array[i] == 1):
      for j in range(len(sig_column)):
        if(perm_mtx[j][i] < sig_column[j]):
          #print(perm_mtx[j][i])
          sig_column[j] = perm_mtx[j][i]

  return sig_column

n = int(input("Insert number n of permutation desired: "))
permutation_matrix = list()
tmp = list(range(0, 10))
array = [0,0,0,0,1,0,0,1,1,0]

for i in range(n):
  #Generate 2 random numbers in order to have different hashing by randomly chaging only these two numbers
  a = np.random.randint(10) 
  b = np.random.randint(10)

  permutation_matrix.append(generatePermutation(tmp, a, b))
  
#[print(*line) for line in permutation_matrix]
  
a = calculateMinhashSignature(permutation_matrix, array)
print(a)


Insert number n of permutation desired: 10
[3, 1, 1, 6, 2, 4, 1, 1, 0, 4]


In [ ]:
def band_splitting(tweet_id, arrayToSplit, b, n):
  tmp = list()
  arrayToReturn = list()
  band_id = 1

  r = int(n/b)
  i = 0
  while i < len(arrayToSplit):
    print(i)
    for j in range(r):
      tmp.append(arrayToSplit[i + j])
    i += r
    print(tmp)
    arrayToReturn.append((tweet_id, (band_id, tmp.copy())))
    band_id += 1
    tmp.clear()

  return arrayToReturn

b = 5
ex = band_splitting("xx", a, b, 10)
ex

0
[3, 1]
2
[1, 6]
4
[2, 4]
6
[1, 1]
8
[0, 4]


[('xx', (1, [3, 1])),
 ('xx', (2, [1, 6])),
 ('xx', (3, [2, 4])),
 ('xx', (4, [1, 1])),
 ('xx', (5, [0, 4]))]

In [ ]:
import hashlib

a = "cnoefvrivpvòslkfdvmkfvpòfvevflàeràòwwwwedweferfeev"

#b = int(hashlib.sha1(str(a).encode('utf-8')).hexdigest(), 16)
b = hashlib.sha1(str(a).encode('utf-8')).hexdigest()
#int(hashlib.sha1(str(a).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)

b

'75c93d49c4b7c23b5ba3abde96216cfe2355bd82'

In [ ]:
#a = [1,2,3,4,5,6,7,8]
a = ['a','b','c','d']

b = ' '.join(str(x) for x in a)

b

'a b c d'

## TO DO:
- Hashing list on shingles
- Verify if the hash is worth in terms of space 